In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

/Users/Tutumm/anaconda3/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata_file = './dataset/song_data.csv'
df_att = pd.read_csv('song_attributes.csv')

In [3]:
song_df_1 = pd.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

In [4]:
song_df_2 =  pd.read_csv(songs_metadata_file)
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

In [5]:
len(song_df.groupby(['user_id']))

76353

In [6]:
songs = song_df['song_id'].unique()
len(songs)

10000

In [7]:
song_df["song"] = song_df["title"].map(str) +' - '+ song_df["artist_name"]
song_transformed = song_df.drop(columns=['title', 'release', 'artist_name', 'year'], axis=0)
song_transformed

,user_id,song_id,listen_count,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly - Foo Fighters
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N' Roll - Héroes del Sile...
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODXRTY12AB0180F3B,1,Paper Gangsta - Lady GaGa
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFGUAY12AB017B0A8,1,Stacked Actors - Foo Fighters
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFRQTD12A81C233C0,1,Sehr kosmisch - Harmonia
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn your eyes - Thievery Corpo...


In [8]:
songs = song_transformed.groupby(['song'])
len(songs)

9953

In [9]:
df = song_transformed.groupby('user_id') \
       .agg({'listen_count':'mean'}) \
       .rename(columns={'listen_count':'mean_count'}) \
       .reset_index()
df['user_unique_id'] = df.index

In [10]:
song_merged = song_transformed.merge(df, left_on='user_id', right_on='user_id', how='inner')
song_merged

,user_id,song_id,listen_count,song,mean_count,user_unique_id
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove - Jack Johnson,1.577778,54961
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas - Paco De Lucia,1.577778,54961
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger - Kanye West,1.577778,54961
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations - Jack Johnson,1.577778,54961
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly - Foo Fighters,1.577778,54961
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N' Roll - Héroes del Sile...,1.577778,54961
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODXRTY12AB0180F3B,1,Paper Gangsta - Lady GaGa,1.577778,54961
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFGUAY12AB017B0A8,1,Stacked Actors - Foo Fighters,1.577778,54961
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFRQTD12A81C233C0,1,Sehr kosmisch - Harmonia,1.577778,54961
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn your eyes - Thievery Corpo...,1.577778,54961


In [11]:
songs = song_merged.groupby(['song'])
len(songs)

9953

In [12]:
df1 = song_transformed.groupby('song') \
       .agg({'listen_count':'mean'}) \
       .rename(columns={'listen_count':'mean_count'}) \
       .reset_index()
df1['song_unique_id'] = df1.index

In [13]:
df1 = df1.drop(['mean_count'], axis=1)

In [14]:
song_merged = song_merged.merge(df1, left_on='song', right_on='song', how='outer')

In [15]:
df = song_merged.drop(['user_id', 'song_id'], axis=1)

In [16]:
df['rating'] = (df.listen_count / df.mean_count)*100

In [17]:
def rating(x):
    if x > 0 and x <= 30:
        return 1
    elif x > 30 and x <= 50:
        return 2
    elif x > 50 and x <= 100:
        return 3
    elif x > 100 and x <= 120:
        return 4
    else: return 5

In [18]:
df['rating_rescaled'] = df['rating'].apply(lambda x: rating(x))

In [19]:
reader = Reader()

data = Dataset.load_from_df(df[['user_unique_id', 'song_unique_id', 'rating_rescaled']][:100000], reader)
data.split(n_folds=5)

svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

/Users/Tutumm/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/Tutumm/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.3075
MAE:  1.0862
------------
Fold 2
RMSE: 1.3090
MAE:  1.0878
------------
Fold 3
RMSE: 1.3175
MAE:  1.0981
------------
Fold 4
RMSE: 1.3163
MAE:  1.0941
------------
Fold 5
RMSE: 1.3192
MAE:  1.0970
------------
------------
Mean RMSE: 1.3139
Mean MAE : 1.0926
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [1.3075442864384408,
                             1.3090385798094979,
                             1.317482908646179,
                             1.316264112495769,
                             1.3192058300360534],
                            'mae': [1.0862314098911676,
                             1.087812166583626,
                             1.0980524770232132,
                             1.0941398332986476,
                             1.096951385672347]})

In [20]:
from surprise.model_selection import cross_validate
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3177  1.3111  1.3148  1.3215  1.3119  1.3154  0.0038  
MAE (testset)     1.0958  1.0884  1.0920  1.0979  1.0905  1.0929  0.0035  
Fit time          7.53    7.44    7.40    7.37    7.25    7.40    0.09    
Test time         0.16    0.16    0.16    0.19    0.17    0.17    0.01    


{'test_rmse': array([ 1.31772178,  1.31113308,  1.31476498,  1.32151284,  1.31186604]),
 'test_mae': array([ 1.09581898,  1.08838327,  1.09198505,  1.09788535,  1.09048933]),
 'fit_time': (7.53133487701416,
  7.439644813537598,
  7.399808883666992,
  7.366750955581665,
  7.248627185821533),
 'test_time': (0.16225004196166992,
  0.16332173347473145,
  0.16198992729187012,
  0.1914060115814209,
  0.16534996032714844)}

In [21]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.2949  1.3041  1.3151  1.3002  1.3074  1.3145  1.3174  1.3241  1.3021  1.3058  1.3086  0.0085  
MAE (testset)     1.0733  1.0810  1.0896  1.0809  1.0886  1.0896  1.0968  1.0976  1.0764  1.0840  1.0858  0.0077  
Fit time          8.10    8.30    8.26    8.19    8.41    8.25    8.05    8.67    9.40    8.57    8.42    0.38    
Test time         0.08    0.09    0.09    0.18    0.09    0.08    0.08    0.08    0.11    0.20    0.11    0.04    


{'test_rmse': array([ 1.29489541,  1.30406315,  1.31510757,  1.30022932,  1.30744497,
         1.31452527,  1.31739599,  1.32411116,  1.30205154,  1.30579102]),
 'test_mae': array([ 1.07325295,  1.08104398,  1.08962596,  1.08086364,  1.08862276,
         1.08958046,  1.09678251,  1.0975852 ,  1.07636987,  1.08396981]),
 'fit_time': (8.09805417060852,
  8.299272060394287,
  8.261734247207642,
  8.19239068031311,
  8.411745071411133,
  8.245269060134888,
  8.048657894134521,
  8.66868805885315,
  9.403337955474854,
  8.566824913024902),
 'test_time': (0.08081197738647461,
  0.09381604194641113,
  0.0937960147857666,
  0.17977118492126465,
  0.08642816543579102,
  0.08254122734069824,
  0.07976007461547852,
  0.07852792739868164,
  0.1074988842010498,
  0.19960904121398926)}

In [22]:
df_att["song"] = df_att["song_title"].map(str) +' - '+ df_att["artist"]
df_att

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,song
0,0,0.010200,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future,Mask Off - Future
1,1,0.199000,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino,Redbone - Childish Gambino
2,2,0.034400,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future,Xanny Family - Future
3,3,0.604000,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House,Master Of None - Beach House
4,4,0.180000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys,Parallel Lines - Junior Boys
5,5,0.004790,0.804,251333,0.560,0.000000,8,0.1640,-6.682,1,0.1850,85.023,4.0,0.264,1,Sneakin’,Drake,Sneakin’ - Drake
6,6,0.014500,0.739,241400,0.472,0.000007,1,0.2070,-11.204,1,0.1560,80.030,4.0,0.308,1,Childs Play,Drake,Childs Play - Drake
7,7,0.020200,0.266,349667,0.348,0.664000,10,0.1600,-11.609,0,0.0371,144.154,4.0,0.393,1,Gyöngyhajú lány,Omega,Gyöngyhajú lány - Omega
8,8,0.048100,0.603,202853,0.944,0.000000,11,0.3420,-3.626,0,0.3470,130.035,4.0,0.398,1,I've Seen Footage,Death Grips,I've Seen Footage - Death Grips
9,9,0.002080,0.836,226840,0.603,0.000000,7,0.5710,-7.792,1,0.2370,99.994,4.0,0.386,1,Digital Animal,Honey Claws,Digital Animal - Honey Claws


In [23]:
df2 = df_att.groupby('song') \
       .agg({'target':'mean'}) \
       .reset_index()
df2['song_unique_id'] = df2.index

# Merge songs attr and user profiles

In [24]:
df1['song_lower'] = df1.song.str.lower()
df2['song_lower'] = df2.song.str.lower()
song_merged_1 = df1.merge(df2, left_on='song_lower', right_on='song_lower', how='inner')

In [25]:
import difflib 

In [26]:
df3 = pd.DataFrame([[1,'one'],[2,'two'],[3,'three'],[4,'four'],[5,'five'],[6,'six']], columns=['number', 'name'])
df4 = pd.DataFrame([['a','one'],['b','too'],['d','fours'],['e','five']], columns=['letter', 'name'])

df4['name'] = df4['name'].apply(lambda x: difflib.get_close_matches(x, df3['name'])[0])

In [27]:
df3 = df1.drop(['song'], axis=1)

In [28]:
df4 = df2.drop(['song_unique_id', 'target', 'song'], axis=1)

In [29]:
df3['song_lower_1'] = df3['song_lower'].apply(lambda x: 0 if not difflib.get_close_matches(x, df4['song_lower']) else difflib.get_close_matches(x, df3['song_lower'])[0])

In [30]:
df5 = df3.merge(df4)
df['song_lower'] = df.song.str.lower()
df_sim_songs_1 = df.loc[df['song_lower'].isin(df5['song_lower_1'])]

In [31]:
df5['song_lower'].to_csv('song_list.csv', sep='\t', encoding='utf-8')

In [32]:
df_sim_songs_1

,listen_count,song,mean_count,user_unique_id,song_unique_id,rating,rating_rescaled,song_lower
2273,1,Learn To Fly - Foo Fighters,1.577778,54961,4723,63.380282,3,learn to fly - foo fighters
2274,4,Learn To Fly - Foo Fighters,1.611111,5416,4723,248.275862,5,learn to fly - foo fighters
2275,1,Learn To Fly - Foo Fighters,1.087719,43942,4723,91.935484,3,learn to fly - foo fighters
2276,1,Learn To Fly - Foo Fighters,1.532258,54615,4723,65.263158,3,learn to fly - foo fighters
2277,1,Learn To Fly - Foo Fighters,5.416667,65943,4723,18.461538,1,learn to fly - foo fighters
2278,4,Learn To Fly - Foo Fighters,5.818182,45354,4723,68.750000,3,learn to fly - foo fighters
2279,2,Learn To Fly - Foo Fighters,2.740741,75901,4723,72.972973,3,learn to fly - foo fighters
2280,1,Learn To Fly - Foo Fighters,1.461538,67816,4723,68.421053,3,learn to fly - foo fighters
2281,1,Learn To Fly - Foo Fighters,2.291209,9097,4723,43.645084,2,learn to fly - foo fighters
2282,1,Learn To Fly - Foo Fighters,1.375000,61548,4723,72.727273,3,learn to fly - foo fighters


In [33]:
df_att['song_lower'] = df_att.song.str.lower()
df_att

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,song,song_lower
0,0,0.010200,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future,Mask Off - Future,mask off - future
1,1,0.199000,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino,Redbone - Childish Gambino,redbone - childish gambino
2,2,0.034400,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future,Xanny Family - Future,xanny family - future
3,3,0.604000,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House,Master Of None - Beach House,master of none - beach house
4,4,0.180000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys,Parallel Lines - Junior Boys,parallel lines - junior boys
5,5,0.004790,0.804,251333,0.560,0.000000,8,0.1640,-6.682,1,0.1850,85.023,4.0,0.264,1,Sneakin’,Drake,Sneakin’ - Drake,sneakin’ - drake
6,6,0.014500,0.739,241400,0.472,0.000007,1,0.2070,-11.204,1,0.1560,80.030,4.0,0.308,1,Childs Play,Drake,Childs Play - Drake,childs play - drake
7,7,0.020200,0.266,349667,0.348,0.664000,10,0.1600,-11.609,0,0.0371,144.154,4.0,0.393,1,Gyöngyhajú lány,Omega,Gyöngyhajú lány - Omega,gyöngyhajú lány - omega
8,8,0.048100,0.603,202853,0.944,0.000000,11,0.3420,-3.626,0,0.3470,130.035,4.0,0.398,1,I've Seen Footage,Death Grips,I've Seen Footage - Death Grips,i've seen footage - death grips
9,9,0.002080,0.836,226840,0.603,0.000000,7,0.5710,-7.792,1,0.2370,99.994,4.0,0.386,1,Digital Animal,Honey Claws,Digital Animal - Honey Claws,digital animal - honey claws


In [34]:
df_sim_songs_2 = df_att.loc[df_att['song_lower'].isin(df5['song_lower_1'])]
df_sim_songs_2

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,song,song_lower
21,21,0.000240,0.747,307680,0.7400,0.369000,1,0.0995,-4.134,1,0.0323,130.031,4.0,0.770,1,House of Jealous Lovers,The Rapture,House of Jealous Lovers - The Rapture,house of jealous lovers - the rapture
39,39,0.135000,0.777,237187,0.5910,0.000055,1,0.1260,-7.431,0,0.0327,124.111,4.0,0.848,1,Somebody's Watching Me,Rockwell,Somebody's Watching Me - Rockwell,somebody's watching me - rockwell
72,72,0.518000,0.683,155133,0.7080,0.000021,4,0.1750,-6.852,0,0.0289,106.446,4.0,0.967,1,Spooky,Dusty Springfield,Spooky - Dusty Springfield,spooky - dusty springfield
105,105,0.181000,0.511,160907,0.7690,0.000000,4,0.0921,-7.032,1,0.0420,129.657,4.0,0.814,1,Be My Baby,The Ronettes,Be My Baby - The Ronettes,be my baby - the ronettes
111,111,0.403000,0.683,142333,0.7100,0.000000,9,0.0860,-7.445,1,0.1140,128.181,4.0,0.930,1,It's My Party,Lesley Gore,It's My Party - Lesley Gore,it's my party - lesley gore
135,135,0.022700,0.698,248053,0.8040,0.059900,4,0.5660,-6.616,1,0.1110,92.001,4.0,0.786,1,Pussy,Brazilian Girls,Pussy - Brazilian Girls,pussy - brazilian girls
136,136,0.103000,0.772,191160,0.5820,0.000066,4,0.3230,-5.960,0,0.3090,172.030,4.0,0.564,1,Black Mags,The Cool Kids,Black Mags - The Cool Kids,black mags - the cool kids
146,146,0.000021,0.259,350920,0.8470,0.823000,2,0.1330,-6.298,1,0.0332,150.162,4.0,0.682,1,Nothing Ever Happened,Deerhunter,Nothing Ever Happened - Deerhunter,nothing ever happened - deerhunter
195,195,0.613000,0.698,230200,0.8780,0.000016,8,0.5180,-0.307,1,0.3140,182.126,4.0,0.816,1,GodLovesUgly,Atmosphere,GodLovesUgly - Atmosphere,godlovesugly - atmosphere
198,198,0.001920,0.558,301573,0.7400,0.000009,6,0.1290,-6.220,1,0.0448,118.293,4.0,0.214,1,All These Things That I've Done,The Killers,All These Things That I've Done - The Killers,all these things that i've done - the killers


In [35]:
df_merged_sim = df_sim_songs_1.merge(df_sim_songs_2)
df_merged_sim

,listen_count,song,mean_count,user_unique_id,song_unique_id,rating,rating_rescaled,song_lower,Unnamed: 0,acousticness,...,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,1,Drive - Incubus,1.577778,54961,2215,63.380282,3,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
1,1,Drive - Incubus,1.857143,11128,2215,53.846154,3,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
2,1,Drive - Incubus,2.642857,37055,2215,37.837838,2,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
3,1,Drive - Incubus,2.291209,9097,2215,43.645084,2,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
4,3,Drive - Incubus,3.333333,35091,2215,90.000000,3,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
5,1,Drive - Incubus,2.696970,29286,2215,37.078652,2,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
6,1,Drive - Incubus,2.641026,42798,2215,37.864078,2,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
7,1,Drive - Incubus,1.217391,1210,2215,82.142857,3,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
8,8,Drive - Incubus,3.470588,28014,2215,230.508475,5,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus
9,13,Drive - Incubus,9.085714,71778,2215,143.081761,5,drive - incubus,1907,0.0661,...,0.150,-5.474,0,0.0361,90.562,4.0,0.668,0,Drive,Incubus


# Done with df_merged_sim variable and Start with Collaborative Filtering

In [36]:
data = Dataset.load_from_df(df[['user_unique_id', 'song_unique_id', 'rating_rescaled']], reader)

In [37]:
trainset = data.build_full_trainset()

In [38]:
svd.fit(trainset)

In [118]:
df_predict = df5.copy()
df_predict = df_predict.reset_index()

user_id = 76353
df_predict['estimate_Score'] = df_predict['song_unique_id'].apply(lambda x: svd.predict(user_id, x).est)

df_predict = df_predict.sort_values('estimate_Score', ascending=False)
df_predict = df_predict.drop(['song_unique_id', 'index'], axis=1)

# len(df_predict[df_predict['estimate_Score'] >= 4.0])
df_predict = df_predict.sort_values('estimate_Score', ascending=False)


In [119]:
df_predict = df_predict.drop(['song_lower_1'], axis=1)
print(df_predict)

                                           song_lower  estimate_Score
12                         candy rain - soul for real        3.786977
70               you get what you give - new radicals        3.738486
8                           be my baby - the ronettes        3.684637
54                       shooting stars - bag raiders        3.658299
16                      do you love me - the contours        3.569032
58                         spooky - dusty springfield        3.555957
71                             you gotta be - des'ree        3.546627
7                              bam bam - sister nancy        3.512156
44                           my own worst enemy - lit        3.510785
50                            pussy - brazilian girls        3.497035
5                           all my life - k-ci & jojo        3.491697
35                         love song - sara bareilles        3.472573
9                 beautiful disaster - jon mclaughlin        3.422848
32                  

# Evaluation for Collaborative Filtering

In [120]:
df_predict

,song_lower,estimate_Score
12,candy rain - soul for real,3.786977
70,you get what you give - new radicals,3.738486
8,be my baby - the ronettes,3.684637
54,shooting stars - bag raiders,3.658299
16,do you love me - the contours,3.569032
58,spooky - dusty springfield,3.555957
71,you gotta be - des'ree,3.546627
7,bam bam - sister nancy,3.512156
44,my own worst enemy - lit,3.510785
50,pussy - brazilian girls,3.497035


In [121]:
temp = df_sim_songs_1.groupby(['user_unique_id']).count()
temp.sort_values(by=['song'], ascending=False)

,listen_count,song,mean_count,song_unique_id,rating,rating_rescaled,song_lower
user_unique_id,,,,,,,
32542,10,10,10,10,10,10,10
56230,10,10,10,10,10,10,10
15940,9,9,9,9,9,9,9
37228,9,9,9,9,9,9,9
30138,9,9,9,9,9,9,9
36455,8,8,8,8,8,8,8
70386,8,8,8,8,8,8,8
26438,8,8,8,8,8,8,8
19774,8,8,8,8,8,8,8


In [122]:
user_id = 32542
# user_id = 56230

df_predict_eval = df_sim_songs_1.copy()
df_predict_eval = df_predict_eval.reset_index()

df_predict_eval['predicted_rating'] = df_predict_eval['song_unique_id'].apply(lambda x: svd.predict(user_id, x).est)

df_predict_eval = df_predict_eval.sort_values('predicted_rating', ascending=False)
df_predict_eval = df_predict_eval.drop(['song_unique_id', 'index', 'listen_count', 'mean_count', 'rating', 'song_lower'], axis=1)

# len(df_predict[df_predict['estimate_Score'] >= 4.0])
df_predict_eval = df_predict_eval.sort_values('predicted_rating', ascending=False)

In [123]:
eval_predicted = df_predict_eval.loc[df_predict_eval['user_unique_id']==user_id]
eval_predicted = eval_predicted['predicted_rating']
eval_real_rating = df_predict_eval.loc[df_predict_eval['user_unique_id']==user_id]
eval_real_rating = eval_real_rating['rating_rescaled']

In [124]:
eval_predicted

11174    3.302857
10813    3.120602
1288     2.913919
8260     2.712305
16722    2.192993
13133    1.954746
2360     1.691449
9030     1.333107
15795    1.000000
7915     1.000000
Name: predicted_rating, dtype: float64

In [125]:
eval_real_rating

11174    5
10813    3
1288     5
8260     3
16722    1
13133    2
2360     1
9030     2
15795    1
7915     1
Name: rating_rescaled, dtype: int64

In [126]:
def rmse(y_actual, y_predicted):
    return np.sqrt(np.mean((y_predicted-y_actual)**2))

In [127]:
print(rmse(eval_real_rating,eval_predicted))

0.983741397392


# Content-based Recommender

In [128]:
from sklearn.metrics.pairwise import linear_kernel

In [129]:
selected_features = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence']
att_matrix = df_sim_songs_2[selected_features]
cosine_sim = linear_kernel(att_matrix, att_matrix)

In [130]:
df_sim_songs_2 = df_sim_songs_2.reset_index()

In [131]:
indices = pd.Series(df_sim_songs_2.index, index=df_sim_songs_2['song_lower']).drop_duplicates()

In [132]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    dt = np.dtype({'names': ['index', 'name'], 'formats':['float','S10']})
    sim_scores = np.array(sim_scores, dt)
    sim_scores = np.sort(sim_scores, order='name')
    sim_scores = sim_scores[1:11]
    song_indices = [i[0] for i in sim_scores]

    return df_sim_songs_2['song_title'].iloc[song_indices]

In [133]:
get_recommendations('house of jealous lovers - the rapture')

0


39              All My Life
26            Blue in Green
36          End Of The Road
7     Nothing Ever Happened
23           All My Friends
57                     Lump
4             It's My Party
2                    Spooky
3                Be My Baby
65       My Own Worst Enemy
Name: song_title, dtype: object

# Hybrid Recommender

In [143]:
song_2000 = pd.read_csv('dataset/output_2000.csv', header=None)

In [216]:
df_predict = df5.copy()
df_predict = df_predict.reset_index()

# max is 76352
user_id = 76352
df_predict['estimate_Score'] = df_predict['song_unique_id'].apply(lambda x: svd.predict(user_id, x).est)

df_predict = df_predict.sort_values('estimate_Score', ascending=False)
df_predict = df_predict.drop(['song_unique_id', 'index'], axis=1)

# len(df_predict[df_predict['estimate_Score'] >= 4.0])
df_predict = df_predict.sort_values('estimate_Score', ascending=False)
df_predict = df_predict.drop(['song_lower_1'], axis=1)

#features include the estimate score
selected_features = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'estimate_Score']

# merge score with the dataset to make the recommendation
df_merge_score = pd.merge(df_sim_songs_2, df_predict, on="song_lower", how="right")

att_matrix = df_merge_score[selected_features]
cosine_sim = linear_kernel(att_matrix, att_matrix)

df_merge_score = df_merge_score.reset_index()

indices = pd.Series(df_merge_score.index, index=df_merge_score['song_lower']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    dt = np.dtype({'names': ['index', 'name'], 'formats':['float','S10']})
    sim_scores = np.array(sim_scores, dt)
    sim_scores = np.sort(sim_scores, order='name')
    original_simscores = sim_scores
    sim_scores = sim_scores[1:11]
    song_indices = [i[0] for i in sim_scores]
    
    show_recommend = df_merge_score.iloc[song_indices]
    show_recommend = show_recommend.sort_values(by='estimate_Score', ascending=False)

    return show_recommend.loc[:, ['song_title', 'estimate_Score']]

get_recommendations('monster mash - bobby "boris" pickett')

13


,song_title,estimate_Score
4,It's My Party,5.000000
2,Spooky,5.000000
22,Do You Love Me,5.000000
13,Monster Mash,5.000000
3,Be My Baby,4.963642
12,Bam Bam,4.914841
66,My Own Worst Enemy,4.912673
27,Waiting,4.791152
70,Jump,4.596971
6,Black Mags,4.576618


# Hybrid Recommender Evaluation

In [221]:
df_merge_score = pd.merge(df_sim_songs_2, df_predict, on="song_lower", how="right")

In [222]:
df_merge_score

,index,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,...,speechiness,tempo,time_signature,valence,target,song_title,artist,song,song_lower,estimate_Score
0,21,21,0.000240,0.747,307680,0.7400,0.369000,1,0.0995,-4.134,...,0.0323,130.031,4.0,0.770,1,House of Jealous Lovers,The Rapture,House of Jealous Lovers - The Rapture,house of jealous lovers - the rapture,4.678702
1,39,39,0.135000,0.777,237187,0.5910,0.000055,1,0.1260,-7.431,...,0.0327,124.111,4.0,0.848,1,Somebody's Watching Me,Rockwell,Somebody's Watching Me - Rockwell,somebody's watching me - rockwell,4.586065
2,72,72,0.518000,0.683,155133,0.7080,0.000021,4,0.1750,-6.852,...,0.0289,106.446,4.0,0.967,1,Spooky,Dusty Springfield,Spooky - Dusty Springfield,spooky - dusty springfield,5.000000
3,105,105,0.181000,0.511,160907,0.7690,0.000000,4,0.0921,-7.032,...,0.0420,129.657,4.0,0.814,1,Be My Baby,The Ronettes,Be My Baby - The Ronettes,be my baby - the ronettes,4.963642
4,111,111,0.403000,0.683,142333,0.7100,0.000000,9,0.0860,-7.445,...,0.1140,128.181,4.0,0.930,1,It's My Party,Lesley Gore,It's My Party - Lesley Gore,it's my party - lesley gore,5.000000
5,135,135,0.022700,0.698,248053,0.8040,0.059900,4,0.5660,-6.616,...,0.1110,92.001,4.0,0.786,1,Pussy,Brazilian Girls,Pussy - Brazilian Girls,pussy - brazilian girls,4.815388
6,136,136,0.103000,0.772,191160,0.5820,0.000066,4,0.3230,-5.960,...,0.3090,172.030,4.0,0.564,1,Black Mags,The Cool Kids,Black Mags - The Cool Kids,black mags - the cool kids,4.576618
7,146,146,0.000021,0.259,350920,0.8470,0.823000,2,0.1330,-6.298,...,0.0332,150.162,4.0,0.682,1,Nothing Ever Happened,Deerhunter,Nothing Ever Happened - Deerhunter,nothing ever happened - deerhunter,4.608639
8,195,195,0.613000,0.698,230200,0.8780,0.000016,8,0.5180,-0.307,...,0.3140,182.126,4.0,0.816,1,GodLovesUgly,Atmosphere,GodLovesUgly - Atmosphere,godlovesugly - atmosphere,4.808542
9,198,198,0.001920,0.558,301573,0.7400,0.000009,6,0.1290,-6.220,...,0.0448,118.293,4.0,0.214,1,All These Things That I've Done,The Killers,All These Things That I've Done - The Killers,all these things that i've done - the killers,4.656962


In [223]:
eval_hyb = pd.merge(df_merge_score.drop(['index', 'Unnamed: 0', 'song_title', 'artist', 'song'], axis=1), song_2000, left_on="song_lower", right_on=song_2000[0])

In [224]:
eval_hyb.drop(eval_hyb.columns[0], axis=1)

,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,0.747,307680,0.7400,0.369000,1,0.0995,-4.134,1,0.0323,130.031,...,65.265756,42.371491,6.034689,0.998728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.777,237187,0.5910,0.000055,1,0.1260,-7.431,0,0.0327,124.111,...,56.578167,48.268695,61.392756,70.040413,70.564942,56.383323,55.920823,38.805535,25.461568,0.000000
2,0.683,155133,0.7080,0.000021,4,0.1750,-6.852,0,0.0289,106.446,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.511,160907,0.7690,0.000000,4,0.0921,-7.032,1,0.0420,129.657,...,3.038857,1.123449,0.350175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.683,142333,0.7100,0.000000,9,0.0860,-7.445,1,0.1140,128.181,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.698,248053,0.8040,0.059900,4,0.5660,-6.616,1,0.1110,92.001,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.772,191160,0.5820,0.000066,4,0.3230,-5.960,0,0.3090,172.030,...,85.967175,108.026885,76.054570,35.399461,149.779458,160.817184,85.194830,9.678509,10.637735,16.945502
7,0.259,350920,0.8470,0.823000,2,0.1330,-6.298,1,0.0332,150.162,...,77.535657,77.450994,82.392658,87.745968,100.955454,125.808570,118.566504,90.206706,12.051336,2.279713
8,0.698,230200,0.8780,0.000016,8,0.5180,-0.307,1,0.3140,182.126,...,27.310238,24.259658,17.740858,22.723508,27.271039,26.577827,32.285893,26.621090,11.004971,26.378331
9,0.558,301573,0.7400,0.000009,6,0.1290,-6.220,1,0.0448,118.293,...,80.861499,80.562083,79.438047,79.472299,75.918134,64.414529,42.281423,8.155408,0.000000,0.000000


In [231]:
df_predict = df5.copy()
df_predict = df_predict.reset_index()

# max is 76352
user_id = 76352
df_predict['estimate_Score'] = df_predict['song_unique_id'].apply(lambda x: svd.predict(user_id, x).est)

df_predict = df_predict.sort_values('estimate_Score', ascending=False)
df_predict = df_predict.drop(['song_unique_id', 'index'], axis=1)

# len(df_predict[df_predict['estimate_Score'] >= 4.0])
df_predict = df_predict.sort_values('estimate_Score', ascending=False)
df_predict = df_predict.drop(['song_lower_1'], axis=1)

#features include the estimate score
selected_features = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'estimate_Score']

# merge score with the dataset to make the recommendation
df_merge_score = pd.merge(df_sim_songs_2, df_predict, on="song_lower", how="right")

att_matrix = df_merge_score[selected_features]
cosine_sim = linear_kernel(att_matrix, att_matrix)

df_merge_score = df_merge_score.reset_index()

indices = pd.Series(df_merge_score.index, index=df_merge_score['song_lower']).drop_duplicates()

def original_sim(title, cosine_sim=cosine_sim):
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    dt = np.dtype({'names': ['index', 'name'], 'formats':['float','S10']})
    sim_scores = np.array(sim_scores, dt)
    sim_scores = np.sort(sim_scores, order='name')
    original_simscores = sim_scores
    sim_scores = sim_scores[1:11]
    song_indices = [i[0] for i in sim_scores]
    
    show_recommend = df_merge_score.iloc[song_indices]
    show_recommend = show_recommend.sort_values(by='estimate_Score', ascending=False)

    return original_simscores

original_sim('monster mash - bobby "boris" pickett')

13


array([( 58., b'2577177805'), (  4., b'2733363841'), (  2., b'2979174888'),
       (  3., b'3090058946'), ( 66., b'3245995237'), ( 22., b'3315898133'),
       ( 27., b'3454666134'), (  6., b'3671037852'), ( 13., b'3687936671'),
       ( 70., b'3746835558'), ( 12., b'3773721599'), ( 62., b'3837977786'),
       ( 10., b'3840282550'), ( 73., b'3936052703'), ( 63., b'4055117299'),
       ( 60., b'4139864697'), ( 31., b'4162659817'), ( 46., b'4175718984'),
       ( 30., b'4207981639'), ( 24., b'4323973589'), ( 21., b'4392089851'),
       (  8., b'4420762075'), ( 29., b'4449702402'), ( 65., b'4474282991'),
       ( 38., b'4505009931'), ( 67., b'4509100156'), ( 61., b'4516262955'),
       ( 14., b'4528553736'), ( 37., b'4529322021'), ( 28., b'4529705997'),
       ( 55., b'4533431750'), (  1., b'4554940028'), ( 54., b'4594116158'),
       ( 69., b'4607673974'), ( 16., b'4642241633'), ( 11., b'4648655527'),
       ( 64., b'4676040300'), ( 72., b'4695512984'), ( 49., b'4698585751'),
       ( 45.

In [232]:
eval_hyb = pd.read_csv('dataset/eval.csv')
cols = list(eval_hyb)
cols.insert(0, cols.pop(cols.index('song_lower')))
eval_hyb = eval_hyb.ix[:, cols]

selected_features_eval = eval_hyb.columns[2:]
att_matrix = eval_hyb[selected_features_eval]
cosine_sim = linear_kernel(att_matrix, att_matrix)

eval_hyb = eval_hyb.reset_index()

indices = pd.Series(eval_hyb.index, index=eval_hyb['song_lower']).drop_duplicates()

def predicted_sim(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    dt = np.dtype({'names': ['index', 'name'], 'formats':['float','S10']})
    sim_scores = np.array(sim_scores, dt)
    sim_scores = np.sort(sim_scores, order='name')
    predicted_simscores = sim_scores
    sim_scores = sim_scores[1:11]
    song_indices = [i[0] for i in sim_scores]
    
    show_recommend = df_merge_score.iloc[song_indices]
    show_recommend = show_recommend.sort_values(by='estimate_Score', ascending=False)

    return predicted_simscores

predicted_sim('monster mash - bobby "boris" pickett')

/Users/Tutumm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


array([( 58., b'2577110082'), (  4., b'2733358770'), (  2., b'2979125018'),
       (  3., b'3090041654'), ( 66., b'3245976916'), ( 22., b'3315863297'),
       ( 27., b'3454687526'), (  6., b'3670904185'), ( 13., b'3688093629'),
       ( 70., b'3746660373'), ( 12., b'3773673486'), ( 62., b'3837967143'),
       ( 10., b'3840346834'), ( 73., b'3935870082'), ( 63., b'4055083783'),
       ( 60., b'4139577171'), ( 31., b'4162633174'), ( 46., b'4175693046'),
       ( 30., b'4207915222'), ( 24., b'4323666419'), ( 21., b'4392131629'),
       (  8., b'4420790472'), ( 29., b'4449533688'), ( 65., b'4474274746'),
       ( 38., b'4504778977'), ( 67., b'4509147778'), ( 61., b'4516026555'),
       ( 14., b'4528526188'), ( 37., b'4529108292'), ( 28., b'4529732835'),
       ( 55., b'4533353203'), (  1., b'4554694409'), ( 54., b'4593982040'),
       ( 69., b'4607631492'), ( 16., b'4642296323'), ( 11., b'4648589395'),
       ( 64., b'4675942647'), ( 72., b'4695458176'), ( 49., b'4698535151'),
       ( 45.

In [239]:
original_simscores = original_sim('monster mash - bobby "boris" pickett')
original_simscores = [i[1] for i in original_simscores]
original_simscores

13


[b'2577177805',
 b'2733363841',
 b'2979174888',
 b'3090058946',
 b'3245995237',
 b'3315898133',
 b'3454666134',
 b'3671037852',
 b'3687936671',
 b'3746835558',
 b'3773721599',
 b'3837977786',
 b'3840282550',
 b'3936052703',
 b'4055117299',
 b'4139864697',
 b'4162659817',
 b'4175718984',
 b'4207981639',
 b'4323973589',
 b'4392089851',
 b'4420762075',
 b'4449702402',
 b'4474282991',
 b'4505009931',
 b'4509100156',
 b'4516262955',
 b'4528553736',
 b'4529322021',
 b'4529705997',
 b'4533431750',
 b'4554940028',
 b'4594116158',
 b'4607673974',
 b'4642241633',
 b'4648655527',
 b'4676040300',
 b'4695512984',
 b'4698585751',
 b'4708821474',
 b'4763610467',
 b'4789132668',
 b'4800233042',
 b'4801000824',
 b'4813541037',
 b'4825063814',
 b'4832495660',
 b'4956802765',
 b'4970514575',
 b'5030161952',
 b'5056029794',
 b'5129005187',
 b'5145654931',
 b'5145654931',
 b'5206071301',
 b'5210679903',
 b'5233225429',
 b'5339404191',
 b'5365598788',
 b'5411169433',
 b'5562247483',
 b'5605264078',
 b'56746

In [260]:
predicted_simscores = predicted_sim('monster mash - bobby "boris" pickett')
predicted_simscores = [i[1] for i in predicted_simscores]
predicted_simscores

[b'2577110082',
 b'2733358770',
 b'2979125018',
 b'3090041654',
 b'3245976916',
 b'3315863297',
 b'3454687526',
 b'3670904185',
 b'3688093629',
 b'3746660373',
 b'3773673486',
 b'3837967143',
 b'3840346834',
 b'3935870082',
 b'4055083783',
 b'4139577171',
 b'4162633174',
 b'4175693046',
 b'4207915222',
 b'4323666419',
 b'4392131629',
 b'4420790472',
 b'4449533688',
 b'4474274746',
 b'4504778977',
 b'4509147778',
 b'4516026555',
 b'4528526188',
 b'4529108292',
 b'4529732835',
 b'4533353203',
 b'4554694409',
 b'4593982040',
 b'4607631492',
 b'4642296323',
 b'4648589395',
 b'4675942647',
 b'4695458176',
 b'4698535151',
 b'4708664676',
 b'4763624884',
 b'4789168799',
 b'4800135675',
 b'4800886835',
 b'4813427047',
 b'4824985958',
 b'4832337962',
 b'4956741076',
 b'4970470507',
 b'5030134847',
 b'5055938784',
 b'5128929315',
 b'5145563660',
 b'5145563660',
 b'5205828647',
 b'5210590879',
 b'5233150712',
 b'5339408639',
 b'5365542716',
 b'5411120576',
 b'5562130989',
 b'5605070101',
 b'56744

In [270]:
original_simscores = list(map(int, original_simscores))
predicted_simscores = list(map(int, predicted_simscores))
original_simscores = pd.to_numeric(original_simscores)
predicted_simscores = pd.to_numeric(predicted_simscores)
print(rmse(original_simscores/100000, predicted_simscores/100000))

1.24406610605
